In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import numpy as np
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder

ModuleNotFoundError: No module named 'torch'

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import numpy as np
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import os

# Define LeNet-5 architecture
class LeNet5(nn.Module):
    def __init__(self, num_classes=33):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=0)  # C1: 32x32 -> 28x28
        self.pool1 = nn.AvgPool2d(kernel_size=2, stride=2)  # S2: 28x28 -> 14x14
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0)  # C3: 14x14 -> 10x10
        self.pool2 = nn.AvgPool2d(kernel_size=2, stride=2)  # S4: 10x10 -> 5x5
        self.conv3 = nn.Conv2d(16, 120, kernel_size=5, stride=1, padding=0)  # C5: 5x5 -> 1x1
        self.fc1 = nn.Linear(120, 84)  # F6
        self.fc2 = nn.Linear(84, num_classes)  # Output

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = F.relu(self.conv3(x))
        x = x.view(-1, 120)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Function to visualize feature maps
def visualize_feature_maps(model, input_image, layer_name):
    model.eval()
    x = input_image.unsqueeze(0)
    if layer_name == 'conv1':
        x = F.relu(model.conv1(x))
    elif layer_name == 'pool1':
        x = F.relu(model.conv1(x))
        x = model.pool1(x)
    elif layer_name == 'conv2':
        x = F.relu(model.conv1(x))
        x = model.pool1(x)
        x = F.relu(model.conv2(x))
    elif layer_name == 'pool2':
        x = F.relu(model.conv1(x))
        x = model.pool1(x)
        x = F.relu(model.conv2(x))
        x = model.pool2(x)
    feature_maps = x.detach().numpy()[0]
    num_features = feature_maps.shape[0]
    plt.figure(figsize=(15, 5))
    for i in range(min(num_features, 6)):  # Visualize up to 6 feature maps
        plt.subplot(1, 6, i+1)
        plt.imshow(feature_maps[i], cmap='gray')
        plt.axis('off')
    plt.savefig(f'feature_maps_{layer_name}.png')
    plt.close()

# Training function
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10, device='cpu'):
    train_losses, val_losses, train_accs, val_accs = [], [], [], []
    for epoch in range(num_epochs):
        model.train()
        running_loss, correct, total = 0.0, 0, 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        train_loss = running_loss / len(train_loader)
        train_acc = correct / total
        train_losses.append(train_loss)
        train_accs.append(train_acc)

        model.eval()
        val_loss, correct, total = 0.0, 0, 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        val_loss = val_loss / len(val_loader)
        val_acc = correct / total
        val_losses.append(val_loss)
        val_accs.append(val_acc)
        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')
    return train_losses, val_losses, train_accs, val_accs

# Plot loss and accuracy
def plot_metrics(train_losses, val_losses, train_accs, val_accs):
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.title('Loss Curves')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(train_accs, label='Train Accuracy')
    plt.plot(val_accs, label='Validation Accuracy')
    plt.title('Accuracy Curves')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.savefig('training_metrics.png')
    plt.close()

# Plot confusion matrix
def plot_confusion_matrix(model, test_loader, device='cpu'):
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.savefig('confusion_matrix.png')
    plt.close()

# Function to load Tifinagh dataset
def load_tifinagh_dataset(data_dir, batch_size=64):
    transform = transforms.Compose([
        transforms.Grayscale(num_output_channels=1),
        transforms.Resize((32, 32)),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])

    # Define paths for train, validation, and test sets
    train_dir = os.path.join(data_dir, 'train')
    val_dir = os.path.join(data_dir, 'val')
    test_dir = os.path.join(data_dir, 'test')

    # Check if directories exist
    for directory in [train_dir, val_dir, test_dir]:
        if not os.path.exists(directory):
            raise FileNotFoundError(f"Directory {directory} does not exist. Please check the dataset path.")

    # Load datasets
    train_dataset = ImageFolder(train_dir, transform=transform)
    val_dataset = ImageFolder(val_dir, transform=transform)
    test_dataset = ImageFolder(test_dir, transform=transform)

    # Verify number of classes
    if len(train_dataset.classes) != 33:
        raise ValueError(f"Expected 33 classes, but found {len(train_dataset.classes)} classes in the dataset.")

    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader, test_loader

# Main execution
def main():
    # Device configuration
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Load Tifinagh dataset (replace with your dataset path)
    data_dir = 'desktop/newfile'  # Update this to your dataset directory
    try:
        train_loader, val_loader, test_loader = load_tifinagh_dataset(data_dir)
    except Exception as e:
        print(f"Error loading dataset: {e}")
        return

    # Initialize model, loss, and optimizers
    model = LeNet5(num_classes=33).to(device)
    criterion = nn.CrossEntropyLoss()

    # SGD Optimizer
    optimizer_sgd = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    
    # Adam Optimizer
    optimizer_adam = optim.Adam(model.parameters(), lr=0.001)

    # Train with SGD
    print("Training LeNet-5 with SGD...")
    train_losses_sgd, val_losses_sgd, train_accs_sgd, val_accs_sgd = train_model(
        model, train_loader, val_loader, criterion, optimizer_sgd, num_epochs=10, device=device
    )

    # Train with Adam (reset model weights)
    model = LeNet5(num_classes=33).to(device)
    print("Training LeNet-5 with Adam...")
    train_losses_adam, val_losses_adam, train_accs_adam, val_accs_adam = train_model(
        model, train_loader, val_loader, criterion, optimizer_adam, num_epochs=10, device=device
    )

    # Visualize metrics
    plot_metrics(train_losses_adam, val_losses_adam, train_accs_adam, val_accs_adam)

    # Visualize feature maps (using first image from test set)
    first_image, _ = next(iter(test_loader))
    visualize_feature_maps(model, first_image[0], 'conv1')
    visualize_feature_maps(model, first_image[0], 'pool1')
    visualize_feature_maps(model, first_image[0], 'conv2')
    visualize_feature_maps(model, first_image[0], 'pool2')

    # Plot confusion matrix
    plot_confusion_matrix(model, test_loader, device)

if __name__ == '__main__':
    main()

ModuleNotFoundError: No module named 'torch'

In [4]:
pip install torch


error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages system-wide, try apt install
    python3-xyz, where xyz is the package you are trying to
    install.
    
    If you wish to install a non-Debian-packaged Python package,
    create a virtual environment using python3 -m venv path/to/venv.
    Then use path/to/venv/bin/python and path/to/venv/bin/pip. Make
    sure you have python3-full installed.
    
    If you wish to install a non-Debian packaged Python application,
    it may be easiest to use pipx install xyz, which will manage a
    virtual environment for you. Make sure you have pipx installed.
    
    See /usr/share/doc/python3.11/README.venv for more information.

note: If you believe this is a mistake, please contact your Python installation or OS distribution provider. You can override this, at the risk of breaking your Python installation or OS, by passing --break-system-packages.
hint: See PEP 668 for the detai

In [3]:
apt install python3-torch

SyntaxError: invalid syntax (649277900.py, line 1)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import os
from sklearn.metrics import confusion_matrix
import glob

ModuleNotFoundError: No module named 'seaborn'

In [2]:
pip install seaborn

error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages system-wide, try apt install
    python3-xyz, where xyz is the package you are trying to
    install.
    
    If you wish to install a non-Debian-packaged Python package,
    create a virtual environment using python3 -m venv path/to/venv.
    Then use path/to/venv/bin/python and path/to/venv/bin/pip. Make
    sure you have python3-full installed.
    
    If you wish to install a non-Debian packaged Python application,
    it may be easiest to use pipx install xyz, which will manage a
    virtual environment for you. Make sure you have pipx installed.
    
    See /usr/share/doc/python3.11/README.venv for more information.

note: If you believe this is a mistake, please contact your Python installation or OS distribution provider. You can override this, at the risk of breaking your Python installation or OS, by passing --break-system-packages.
hint: See PEP 668 for the detai

In [3]:
pip install python3-seaborn


error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages system-wide, try apt install
    python3-xyz, where xyz is the package you are trying to
    install.
    
    If you wish to install a non-Debian-packaged Python package,
    create a virtual environment using python3 -m venv path/to/venv.
    Then use path/to/venv/bin/python and path/to/venv/bin/pip. Make
    sure you have python3-full installed.
    
    If you wish to install a non-Debian packaged Python application,
    it may be easiest to use pipx install xyz, which will manage a
    virtual environment for you. Make sure you have pipx installed.
    
    See /usr/share/doc/python3.11/README.venv for more information.

note: If you believe this is a mistake, please contact your Python installation or OS distribution provider. You can override this, at the risk of breaking your Python installation or OS, by passing --break-system-packages.
hint: See PEP 668 for the detai

In [4]:
import numpy as np
import os

In [5]:
import numpy as np
import os

# Utility functions
def im2col(input_data, filter_h, filter_w, stride=1, pad=0):
    """Convert image to column matrix for convolution."""
    N, C, H, W = input_data.shape
    out_h = (H + 2 * pad - filter_h) // stride + 1
    out_w = (W + 2 * pad - filter_w) // stride + 1

    img = np.pad(input_data, [(0, 0), (0, 0), (pad, pad), (pad, pad)], 'constant')
    col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))

    for y in range(filter_h):
        y_max = y + stride * out_h
        for x in range(filter_w):
            x_max = x + stride * out_w
            col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]

    col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N * out_h * out_w, -1)
    return col

def col2im(col, input_shape, filter_h, filter_w, stride=1, pad=0):
    """Convert column matrix back to image."""
    N, C, H, W = input_shape
    out_h = (H + 2 * pad - filter_h) // stride + 1
    out_w = (W + 2 * pad - filter_w) // stride + 1
    col = col.reshape(N, out_h, out_w, C, filter_h, filter_w).transpose(0, 3, 4, 5, 1, 2)
    img = np.zeros((N, C, H + 2 * pad, W + 2 * pad))
    for y in range(filter_h):
        y_max = y + stride * out_h
        for x in range(filter_w):
            x_max = x + stride * out_w
            img[:, :, y:y_max:stride, x:x_max:stride] += col[:, :, y, x, :, :]
    return img[:, :, pad:H + pad, pad:W + pad]

# Layer implementations
def conv_layer(x, W, b, stride=1, pad=0):
    """Convolution layer: x (N, C, H, W), W (F, C, FH, FW), b (F)."""
    N, C, H, W = x.shape
    F, _, FH, FW = W.shape
    out_h = (H + 2 * pad - FH) // stride + 1
    out_w = (W + 2 * pad - FW) // stride + 1

    col = im2col(x, FH, FW, stride, pad)
    col_W = W.reshape(F, -1).T
    out = np.dot(col, col_W) + b
    out = out.reshape(N, out_h, out_w, F).transpose(0, 3, 1, 2)
    return out, (x, col)

def conv_layer_backward(dout, cache, W, stride=1, pad=0):
    """Backward pass for convolution."""
    x, col = cache
    F, C, FH, FW = W.shape
    N, _, out_h, out_w = dout.shape

    db = np.sum(dout, axis=(0, 2, 3))
    dout = dout.transpose(0, 2, 3, 1).reshape(-1, F)
    col_W = W.reshape(F, -1).T
    dcol = np.dot(dout, col_W.T)
    dW = np.dot(col.T, dout).reshape(W.shape)
    dx = col2im(dcol, x.shape, FH, FW, stride, pad)
    return dx, dW, db

def relu(x):
    """ReLU activation."""
    return np.maximum(0, x), x

def relu_backward(dout, cache):
    """Backward pass for ReLU."""
    return dout * (cache > 0)

def avg_pool(x, kernel_size, stride):
    """Average pooling layer."""
    N, C, H, W = x.shape
    out_h = (H - kernel_size) // stride + 1
    out_w = (W - kernel_size) // stride + 1

    col = im2col(x, kernel_size, kernel_size, stride, 0)
    col = col.reshape(-1, kernel_size * kernel_size)
    out = np.mean(col, axis=1)
    out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)
    return out, (x, kernel_size, stride)

def avg_pool_backward(dout, cache):
    """Backward pass for average pooling."""
    x, kernel_size, stride = cache
    N, C, H, W = x.shape
    out_h = (H - kernel_size) // stride + 1
    out_w = (W - kernel_size) // stride + 1

    dout = dout.transpose(0, 2, 3, 1).reshape(-1, C)
    dcol = np.repeat(dout / (kernel_size * kernel_size), kernel_size * kernel_size, axis=1)
    dx = col2im(dcol, x.shape, kernel_size, kernel_size, stride, 0)
    return dx

def fc_layer(x, W, b):
    """Fully connected layer."""
    out = np.dot(x, W) + b
    return out, (x,)

def fc_layer_backward(dout, cache, W):
    """Backward pass for fully connected layer."""
    x, = cache
    db = np.sum(dout, axis=0)
    dW = np.dot(x.T, dout)
    dx = np.dot(dout, W.T)
    return dx, dW, db

def softmax(x):
    """Softmax activation."""
    exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

def cross_entropy_loss(pred, y):
    """Cross-entropy loss."""
    N = pred.shape[0]
    log_pred = -np.log(pred[np.arange(N), y] + 1e-10)
    loss = np.sum(log_pred) / N
    grad = pred.copy()
    grad[np.arange(N), y] -= 1
    grad /= N
    return loss, grad

# LeNet-5 model
class LeNet5:
    def __init__(self, num_classes=33):
        # Initialize weights and biases
        self.params = {}
        # C1: 1 input, 6 filters, 5x5
        self.params['W1'] = np.random.randn(6, 1, 5, 5) * 0.01
        self.params['b1'] = np.zeros(6)
        # C3: 6 inputs, 16 filters, 5x5
        self.params['W2'] = np.random.randn(16, 6, 5, 5) * 0.01
        self.params['b2'] = np.zeros(16)
        # C5: 16 inputs, 120 filters, 5x5
        self.params['W3'] = np.random.randn(120, 16, 5, 5) * 0.01
        self.params['b3'] = np.zeros(120)
        # F6: 120 -> 84
        self.params['W4'] = np.random.randn(120, 84) * 0.01
        self.params['b4'] = np.zeros(84)
        # Output: 84 -> 33
        self.params['W5'] = np.random.randn(84, num_classes) * 0.01
        self.params['b5'] = np.zeros(num_classes)

    def forward(self, x):
        """Forward pass."""
        cache = {}
        # C1: 32x32x1 -> 28x28x6
        out, cache['conv1'] = conv_layer(x, self.params['W1'], self.params['b1'])
        out, cache['relu1'] = relu(out)
        # S2: 28x28x6 -> 14x14x6
        out, cache['pool1'] = avg_pool(out, kernel_size=2, stride=2)
        # C3: 14x14x6 -> 10x10x16
        out, cache['conv2'] = conv_layer(out, self.params['W2'], self.params['b2'])
        out, cache['relu2'] = relu(out)
        # S4: 10x10x16 -> 5x5x16
        out, cache['pool2'] = avg_pool(out, kernel_size=2, stride=2)
        # C5: 5x5x16 -> 1x1x120
        out, cache['conv3'] = conv_layer(out, self.params['W3'], self.params['b3'])
        out, cache['relu3'] = relu(out)
        # Flatten: 1x1x120 -> 120
        out = out.reshape(out.shape[0], -1)
        # F6: 120 -> 84
        out, cache['fc1'] = fc_layer(out, self.params['W4'], self.params['b4'])
        out, cache['relu4'] = relu(out)
        # Output: 84 -> 33
        out, cache['fc2'] = fc_layer(out, self.params['W5'], self.params['b5'])
        # Softmax
        scores = softmax(out)
        cache['scores'] = scores
        return scores, cache

    def backward(self, dout, cache):
        """Backward pass."""
        grads = {}
        # Output layer
        dx, grads['W5'], grads['b5'] = fc_layer_backward(dout, cache['fc2'], self.params['W5'])
        dx = relu_backward(dx, cache['relu4'])
        # F6
        dx, grads['W4'], grads['b4'] = fc_layer_backward(dx, cache['fc1'], self.params['W4'])
        dx = dx.reshape(-1, 120, 1, 1)
        # C5
        dx = relu_backward(dx, cache['relu3'])
        dx, grads['W3'], grads['b3'] = conv_layer_backward(dx, cache['conv3'], self.params['W3'])
        # S4
        dx = avg_pool_backward(dx, cache['pool2'])
        # C3
        dx = relu_backward(dx, cache['relu2'])
        dx, grads['W2'], grads['b2'] = conv_layer_backward(dx, cache['conv2'], self.params['W2'])
        # S2
        dx = avg_pool_backward(dx, cache['pool1'])
        # C1
        dx = relu_backward(dx, cache['relu1'])
        dx, grads['W1'], grads['b1'] = conv_layer_backward(dx, cache['conv1'], self.params['W1'])
        return grads

# Optimizer implementations
class SGD:
    def __init__(self, params, lr=0.01, momentum=0.9):
        self.lr = lr
        self.momentum = momentum
        self.velocity = {k: np.zeros_like(v) for k, v in params.items()}

    def step(self, grads, params):
        for key in params:
            self.velocity[key] = self.momentum * self.velocity[key] - self.lr * grads[key]
            params[key] += self.velocity[key]

class Adam:
    def __init__(self, params, lr=0.001, beta1=0.9, beta2=0.999, epsilon=1e-8):
        self.lr = lr
        self.beta1 = beta1
        self.beta2 = beta2
        self.epsilon = epsilon
        self.m = {k: np.zeros_like(v) for k, v in params.items()}
        self.v = {k: np.zeros_like(v) for k, v in params.items()}
        self.t = 0

    def step(self, grads, params):
        self.t += 1
        for key in params:
            self.m[key] = self.beta1 * self.m[key] + (1 - self.beta1) * grads[key]
            self.v[key] = self.beta2 * self.v[key] + (1 - self.beta2) * (grads[key] ** 2)
            m_hat = self.m[key] / (1 - self.beta1 ** self.t)
            v_hat = self.v[key] / (1 - self.beta2 ** self.t)
            params[key] -= self.lr * m_hat / (np.sqrt(v_hat) + self.epsilon)

# Dataset loading
def load_tifinagh_dataset(data_dir, split='train'):
    """Load Tifinagh dataset from .npy files."""
    data = []
    labels = []
    class_dirs = sorted(os.listdir(os.path.join(data_dir, split)))
    if len(class_dirs) != 33:
        raise ValueError(f"Expected 33 classes, found {len(class_dirs)} in {split} set.")

    for class_idx, class_name in enumerate(class_dirs):
        class_path = os.path.join(data_dir, split, class_name)
        for img_path in [f for f in os.listdir(class_path) if f.endswith('.npy')]:
            img = np.load(os.path.join(class_path, img_path))
            if img.shape != (32, 32):
                raise ValueError(f"Image {img_path} has incorrect shape {img.shape}, expected (32, 32)")
            img = (img / 255.0 - 0.5) / 0.5  # Normalize to [-1, 1]
            data.append(img)
            labels.append(class_idx)

    data = np.array(data)[:, np.newaxis, :, :]  # Shape: (N, 1, 32, 32)
    labels = np.array(labels)
    return data, labels

# Training function
def train_model(model, train_data, train_labels, val_data, val_labels, optimizer, num_epochs=10, batch_size=64):
    train_losses, val_losses, train_accs, val_accs = [], [], [], []
    N = train_data.shape[0]

    for epoch in range(num_epochs):
        # Shuffle training data
        indices = np.random.permutation(N)
        train_data = train_data[indices]
        train_labels = train_labels[indices]

        # Training
        running_loss, correct, total = 0.0, 0, 0
        for i in range(0, N, batch_size):
            batch_data = train_data[i:i + batch_size]
            batch_labels = train_labels[i:i + batch_size]
            scores, cache = model.forward(batch_data)
            loss, dout = cross_entropy_loss(scores, batch_labels)
            grads = model.backward(dout, cache)
            optimizer.step(grads, model.params)
            running_loss += loss * batch_data.shape[0]
            predictions = np.argmax(scores, axis=1)
            correct += np.sum(predictions == batch_labels)
            total += batch_data.shape[0]

        train_loss = running_loss / N
        train_acc = correct / total
        train_losses.append(train_loss)
        train_accs.append(train_acc)

        # Validation
        val_scores, _ = model.forward(val_data)
        val_loss, _ = cross_entropy_loss(val_scores, val_labels)
        val_predictions = np.argmax(val_scores, axis=1)
        val_acc = np.sum(val_predictions == val_labels) / len(val_labels)
        val_losses.append(val_loss)
        val_accs.append(val_acc)

        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, '
              f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')

    return train_losses, val_losses, train_accs, val_accs

# Visualization functions
def save_metrics(train_losses, val_losses, train_accs, val_accs):
    """Save loss and accuracy curves as CSV."""
    with open('training_metrics.csv', 'w') as f:
        f.write('Epoch,Train Loss,Val Loss,Train Acc,Val Acc\n')
        for i in range(len(train_losses)):
            f.write(f'{i+1},{train_losses[i]},{val_losses[i]},{train_accs[i]},{val_accs[i]}\n')

def save_confusion_matrix(model, test_data, test_labels):
    """Save confusion matrix as NumPy array."""
    scores, _ = model.forward(test_data)
    predictions = np.argmax(scores, axis=1)
    cm = np.zeros((33, 33), dtype=int)
    for t, p in zip(test_labels, predictions):
        cm[t, p] += 1
    np.save('confusion_matrix.npy', cm)

def save_feature_maps(model, input_image, layer_name):
    """Save feature maps as NumPy arrays."""
    input_image = input_image[np.newaxis, np.newaxis, :, :]  # Shape: (1, 1, 32, 32)
    out = input_image
    if layer_name == 'conv1':
        out, _ = conv_layer(out, model.params['W1'], model.params['b1'])
        out, _ = relu(out)
    elif layer_name == 'pool1':
        out, _ = conv_layer(out, model.params['W1'], model.params['b1'])
        out, _ = relu(out)
        out, _ = avg_pool(out, kernel_size=2, stride=2)
    elif layer_name == 'conv2':
        out, _ = conv_layer(out, model.params['W1'], model.params['b1'])
        out, _ = relu(out)
        out, _ = avg_pool(out, kernel_size=2, stride=2)
        out, _ = conv_layer(out, model.params['W2'], model.params['b2'])
        out, _ = relu(out)
    elif layer_name == 'pool2':
        out, _ = conv_layer(out, model.params['W1'], model.params['b1'])
        out, _ = relu(out)
        out, _ = avg_pool(out, kernel_size=2, stride=2)
        out, _ = conv_layer(out, model.params['W2'], model.params['b2'])
        out, _ = relu(out)
        out, _ = avg_pool(out, kernel_size=2, stride=2)

    np.save(f'feature_maps_{layer_name}.npy', out[0])

# Main execution
def main():
    # Load dataset
    data_dir = 'Desktop\amhcd-data-64'  # Replace with actual path
    try:
        train_data, train_labels = load_tifinagh_dataset(data_dir, 'train')
        val_data, val_labels = load_tifinagh_dataset(data_dir, 'val')
        test_data, test_labels = load_tifinagh_dataset(data_dir, 'test')
    except Exception as e:
        print(f"Error loading dataset: {e}")
        return

    # Initialize model
    model = LeNet5(num_classes=33)

    # Train with Adam
    print("Training with Adam...")
    adam_optimizer = Adam(model.params, lr=0.001)
    train_losses_adam, val_losses_adam, train_accs_adam, val_accs_adam = train_model(
        model, train_data, train_labels, val_data, val_labels, adam_optimizer
    )

    # Save metrics
    save_metrics(train_losses_adam, val_losses_adam, train_accs_adam, val_accs_adam)

    # Save feature maps (using first test image)
    first_image = test_data[0]
    save_feature_maps(model, first_image, 'conv1')
    save_feature_maps(model, first_image, 'pool1')
    save_feature_maps(model, first_image, 'conv2')
    save_feature_maps(model, first_image, 'pool2')

    # Save confusion matrix
    save_confusion_matrix(model, test_data, test_labels)

if __name__ == '__main__':
    main()

Error loading dataset: [Errno 2] No such file or directory: 'Desktop\x07mhcd-data-64/train'
